## Causal-LLM 因果语言模型训练实例

### Step1 导入相关包

In [1]:
import os

# 设置可见的 GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "4,5,7"

from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, TrainingArguments, Trainer

### Step2 加载数据集

In [2]:
ds = load_dataset("pleisto/wikipedia-cn-20230720-filtered", split="train[:10000]")
ds

Dataset({
    features: ['completion', 'source'],
    num_rows: 10000
})

In [3]:
ds[0]

{'completion': '昭通机场（ZPZT）是位于中国云南昭通的民用机场，始建于1935年，1960年3月开通往返航班“昆明－昭通”，原来属军民合用机场。1986年机场停止使用。1991年11月扩建，于1994年2月恢复通航。是西南地区「文明机场」，通航城市昆明。 机场占地1957亩，飞行区等级为4C，有一条跑道，长2720米，宽48米，可供波音737及以下机型起降。机坪面积6600平方米，停机位2个，航站楼面积1900平方米。位于城东6公里处，民航路与金鹰大道交叉处。\n航点\n客服电话\n昭通机场客服电话：0870-2830004',
 'source': 'wikipedia.zh2307'}

### Step3 数据集处理

In [4]:
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-389m-zh")

In [5]:
def process_function(examples):
    # 给每句话加上eos_token，让模型知道什么时候结束
    contents = [e + tokenizer.eos_token for e in examples["completion"]]
    return tokenizer(contents, truncation=True, max_length=384)

In [6]:
tokenized_ds = ds.map(process_function, batched=True, remove_columns=ds.column_names)
tokenized_ds

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 10000
})

In [7]:
from torch.utils.data import DataLoader

dl = DataLoader(tokenized_ds, batch_size=2, collate_fn=DataCollatorForLanguageModeling(tokenizer, mlm=False))
dl

In [8]:
next(enumerate(dl))  # attention_mask中的非-100值代表被mask的token，即input_ids里的103

(0,
 {'input_ids': tensor([[    3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3

In [9]:
tokenizer.pad_token, tokenizer.pad_token_id

('<pad>', 3)

In [11]:
tokenizer.eos_token, tokenizer.eos_token_id

('</s>', 2)

### Step4 创建模型

In [12]:
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-389m-zh")

config.json:   0%|          | 0.00/431 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

### Step5 配置训练参数

In [13]:
import logging

logging.basicConfig(level=logging.INFO)

In [14]:
args = TrainingArguments(
    output_dir="./causal_lm",
    per_device_train_batch_size=32,
    logging_steps=10,
    num_train_epochs=1
)

### Step6 创建训练器

In [16]:
trainer = Trainer(
    args=args,
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

### Step7 模型训练

In [17]:
trainer.train()

/node6_1/tanshuai/.conda/envs/abc/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
10,3.928700
20,3.744900
30,3.600500
40,3.636900
50,3.598600
60,3.524000
70,3.533900
80,3.455900
90,3.486100
100,3.506600


TrainOutput(global_step=105, training_loss=3.595743124825614, metrics={'train_runtime': 233.7499, 'train_samples_per_second': 42.781, 'train_steps_per_second': 0.449, 'total_flos': 6965302394880000.0, 'train_loss': 3.595743124825614, 'epoch': 1.0})

### Step8 模型推理

In [18]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

In [23]:
pipe("昭通机场（ZPZT）是位于中国云南昭通的民用", max_length=128, do_sample=True)

[{'generated_text': '昭通机场（ZPZT）是位于中国云南昭通的民用机场，始建于1958年，1987年随昭北机场建成通飞机起降，2000年机场改建为货运机场，2003年机场改建为民用机场，2008年机场扩建改造工程进入尾声。\n航线\n咸阳站是咸阳市的枢纽航空航点，也是咸阳主城区的集交会点，机场分属咸阳主城区的咸阳经济开发区、咸阳客运枢纽机场。\n其他机场\n* 咸阳经济开发区\n* 咸阳客运枢纽机场\n* 咸阳经济技术开发区机场\n* 咸阳'}]

In [29]:
pipe("结构化剪枝是一种深度学习模型压缩", max_length=128, do_sample=True)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


[{'generated_text': '结构化剪枝是一种深度学习模型压缩机制，能够产生许多剪枝并保持剪枝的深度一致。深度学习是一种基于特征学习的深度融合函数，特征就是用于识别并识别对象的可识别值。\n深度学习模型可以产生多个剪枝，而当参数向量得到足够多的时候，剪枝的数量就会比原来的数少，因此，深度学习模型可以压缩图像，减少剪枝，并提高图像的质量。\n生成化剪枝\n生成化剪枝是图像序列（图像序列是剪枝数量，而不是剪枝大小）的一部份，生成化剪枝可以作为图像特征值'}]

In [30]:
pipe("下面是一则游戏新闻。小编报道，近日，游戏产业发展的非常", max_length=128, do_sample=True)

[{'generated_text': '下面是一则游戏新闻。小编报道，近日，游戏产业发展的非常好的地方——日本任天堂推出了一款基于掌机掌阅智能系统PlayStation 3的智能手机游戏——Pokemon Go，并宣称该款游戏可以玩到《Pokemon》这个系列的全部游戏了。\n游戏制作\n日本任天堂和松下公司合作开发了与掌阅智能系统PlayStation 3的智能手机游戏，不过任天堂方面对游戏内容的宣传并没有披露相关内容。Pokemon Go由任天堂和松下公司联手开发的。任天堂向《Pokemon Go》开发的团队，如Pokemon Go的开发人员——Kazaki'}]